<a href="https://colab.research.google.com/github/donghyundavidchoi/Data_Analysis/blob/main/%EB%AA%A8%EB%8D%B8%EB%A7%81/%EC%A7%91%EA%B0%92%EC%98%88%EC%B8%A1%EB%AA%A8%EB%8D%B8_RandomizedSearchCV_RandomForestRegressor_RMSE_merge_concat_select_dtypes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 집값 예측 모델

In [1]:
!gdown --fuzzy https://drive.google.com/file/d/1txRVXp-Eh222PzIeTQ-fuAoUYFR9f7q-/view?usp=sharing
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Downloading...
From: https://drive.google.com/uc?id=1txRVXp-Eh222PzIeTQ-fuAoUYFR9f7q-
To: /content/kaggle.json
100% 73.0/73.0 [00:00<00:00, 100kB/s]


In [2]:
!kaggle competitions download -c house-prices-advanced-regression-techniques

house-prices-advanced-regression-techniques.zip: Skipping, found more recently modified local copy (use --force to force download)


In [48]:
!unzip /content/house-prices-advanced-regression-techniques.zip

Archive:  /content/house-prices-advanced-regression-techniques.zip
replace data_description.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: data_description.txt    
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [218]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):

    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2022)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("/content/train.csv")
df = df.drop(columns=['Id'])
X_train, X_test, y_train, y_test = exam_data_load(df, target='SalePrice')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1168, 80), (292, 80), (1168, 2), (292, 2))

-------

# EDA

In [219]:
X_train

,id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
891,891,60,RL,70.0,11184,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,7,2009,WD,Normal
266,266,60,RL,70.0,11207,Pave,NaN,IR1,HLS,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1368,1368,120,RM,NaN,4435,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2009,WD,Normal
162,162,20,RL,95.0,12182,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2010,New,Partial
639,639,120,RL,53.0,3982,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,10,2006,New,Partial
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,240,20,FV,75.0,9000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal
624,624,60,RL,80.0,10400,Pave,NaN,Reg,Lvl,AllPub,...,147,0,NaN,NaN,NaN,0,11,2006,WD,Normal
173,173,20,RL,80.0,10197,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2008,COD,Normal
1244,1244,70,RL,NaN,11435,Pave,NaN,IR1,HLS,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal


In [220]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1168 entries, 891 to 893
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1168 non-null   int64  
 1   MSSubClass     1168 non-null   int64  
 2   MSZoning       1168 non-null   object 
 3   LotFrontage    959 non-null    float64
 4   LotArea        1168 non-null   int64  
 5   Street         1168 non-null   object 
 6   Alley          75 non-null     object 
 7   LotShape       1168 non-null   object 
 8   LandContour    1168 non-null   object 
 9   Utilities      1168 non-null   object 
 10  LotConfig      1168 non-null   object 
 11  LandSlope      1168 non-null   object 
 12  Neighborhood   1168 non-null   object 
 13  Condition1     1168 non-null   object 
 14  Condition2     1168 non-null   object 
 15  BldgType       1168 non-null   object 
 16  HouseStyle     1168 non-null   object 
 17  OverallQual    1168 non-null   int64  
 18  Overall

In [221]:
X_train.describe()

,id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1168.000000,1168.000000,959.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1161.000000,1168.000000,...,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000
mean,735.149829,56.185788,70.277372,10482.439212,6.080479,5.579623,1971.089041,1984.604452,103.852713,451.387842,...,473.804795,92.162671,46.104452,22.242295,3.761130,15.142123,1.903253,36.070205,6.306507,2007.791952
std,424.013242,41.506369,24.767651,9729.176951,1.391748,1.118741,30.190295,20.726175,181.793075,459.024051,...,212.930995,125.228453,65.269721,59.992862,31.311799,57.180410,32.691464,312.531832,2.691870,1.323586
min,0.000000,20.000000,21.000000,1477.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,374.750000,20.000000,59.500000,7585.250000,5.000000,5.000000,1954.000000,1966.000000,0.000000,0.000000,...,336.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000
50%,735.500000,50.000000,70.000000,9500.000000,6.000000,5.000000,1972.000000,1993.000000,0.000000,395.000000,...,480.000000,0.000000,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,1103.250000,70.000000,80.000000,11600.000000,7.000000,6.000000,2000.000000,2004.000000,168.000000,728.750000,...,576.000000,168.000000,66.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,1459.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1378.000000,5644.000000,...,1418.000000,736.000000,547.000000,386.000000,508.000000,480.000000,648.000000,8300.000000,12.000000,2010.000000


In [222]:
for i in X_train.columns:
    if X_train[i].isna().sum() != 0:
        print('{} : {}'.format(i, X_train[i].isna().sum()))

LotFrontage : 209
Alley : 1093
MasVnrType : 7
MasVnrArea : 7
BsmtQual : 31
BsmtCond : 31
BsmtExposure : 32
BsmtFinType1 : 31
BsmtFinType2 : 32
Electrical : 1
FireplaceQu : 564
GarageType : 64
GarageYrBlt : 64
GarageFinish : 64
GarageQual : 64
GarageCond : 64
PoolQC : 1164
Fence : 946
MiscFeature : 1125


In [223]:
for i in X_train.columns:
    if X_train[i].isna().sum() != 0:
        print('{} : {}'.format(i, X_train[i].nunique()))

LotFrontage : 106
Alley : 2
MasVnrType : 4
MasVnrArea : 282
BsmtQual : 4
BsmtCond : 4
BsmtExposure : 4
BsmtFinType1 : 6
BsmtFinType2 : 6
Electrical : 5
FireplaceQu : 5
GarageType : 6
GarageYrBlt : 96
GarageFinish : 3
GarageQual : 5
GarageCond : 5
PoolQC : 2
Fence : 4
MiscFeature : 4


---------

# 결측치 처리 계획
LotFrontage : LotArea에 영향을 받을 것. LotShape 별로 LotArea에 비례하여 결측치 처리.   
실제로는 행 자체를 드롭하는 것이 현명하나 연습용인 만큼 결측치 처리 시도.   
Alley : 결측치의 경우 뒷골목으로 향하는 길이 없다는 뜻. 별도 표기로 결측치 처리.   
MasVnrType : 건물 외벽 유무. 결측치는 별도 외벽 장식이 없다는 뜻.   
MasVnrArea : 건물 외벽 면적. 결측치의 경우 별도 외벽 장식이 없다는 뜻.   
Bsmt 관련 열 : 숫자가 다 같지 않고 다른 행이 있음. 하나만 결측인 경우 해당 결측행 삭제.    
Electrical : 전력시스템. 결측치가 하나만 있으므로 삭제.   
FireplaceQu : 결측치는 Fireplace가 없다는 뜻.   
Garage 관련 열 : 결측치는 Garage가 없다는 뜻.   
지하실과 마찬가지로 모든 결측치가 같은 집인지 확인해보고 결측치 처리 진행.   
PoolQC : 결측치는 수영장이 없다는 뜻.   
Fence : 결측치는 울타리가 없다는 뜻.   
MiscFeature : 별도 카테고리가 없는 특징. 결측치는 별도의 특징이 없다는 뜻.   

-------

# LotShape별 평균 LotFrontage와 LotArea의 비율을 구하고 이를 적용하여 LotFrontage의 결측치 채우기

In [224]:
LotArea_LotFrontage_Ratio = X_train.groupby(['LotShape']).apply(lambda x : x['LotFrontage'].mean() / x['LotArea'].mean()).to_frame()
LotArea_LotFrontage_Ratio.reset_index(level=['LotShape'], inplace = True)

In [225]:
LotArea_LotFrontage_Ratio = LotArea_LotFrontage_Ratio.rename(columns = {0 : 'Ratio'})

In [226]:
LotArea_LotFrontage_Ratio

,LotShape,Ratio
0,IR1,0.006693
1,IR2,0.002982
2,IR3,0.003265
3,Reg,0.007570


In [227]:
X_train = pd.merge(X_train, LotArea_LotFrontage_Ratio, how='left', on='LotShape')

In [228]:
X_train['Ratio']

0       0.007570
1       0.006693
2       0.007570
3       0.007570
4       0.007570
          ...   
1163    0.007570
1164    0.007570
1165    0.006693
1166    0.006693
1167    0.007570
Name: Ratio, Length: 1168, dtype: float64

In [229]:
X_train['LotFrontage'] = X_train['LotFrontage'].fillna(X_train['LotArea'] * X_train['Ratio'])

In [230]:
X_train['LotFrontage'].isna().sum()

0

LotFrontage가 제대로 채워진 것을 확인

---------

# 관련 데이터가 없는 경우 결측치로 처리되어 있음.   
# 만약 같은 카테고리에 걸쳐 결측치로 기록되어 있지 않고 다른 이상치가 있다면 삭제하는 것이 바람직할 것.   
# 같은 카테고리에 걸쳐 있지 않은 결측행을 검출하는 알고리즘 적용   

In [231]:
drop_index = []

In [232]:
cols = ['MasVnrType', 'MasVnrArea']
temp3 = []
for i in range(len(cols)):
    if i+1 == len(cols):
        print(temp3)
        break
    temp1 = X_train[cols[i]][X_train[cols[i]].isna()].index
    temp2 = X_train[cols[i+1]][X_train[cols[i+1]].isna()].index
    for i in temp2:
        if i not in temp1:
            temp3.append(i)
            drop_index.append(i)

[]


In [233]:
cols = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']
temp3 = []
for i in range(len(cols)):
    if i+1 == len(cols):
        print(temp3)
        break
    temp1 = X_train[cols[i]][X_train[cols[i]].isna()].index
    temp2 = X_train[cols[i+1]][X_train[cols[i+1]].isna()].index
    for i in temp2:
        if i not in temp1:
            temp3.append(i)
            drop_index.append(i)

[656, 372]


In [234]:
cols = ['GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond']
temp3 = []
for i in range(len(cols)):
    if i+1 == len(cols):
        print(temp3)
        break
    temp1 = X_train[cols[i]][X_train[cols[i]].isna()].index
    temp2 = X_train[cols[i+1]][X_train[cols[i+1]].isna()].index
    for i in temp2:
        if i not in temp1:
            temp3.append(i)
            drop_index.append(i)

[]


In [235]:
drop_index

[656, 372]

------------

# 결측치가 하나만 있는 Electrical의 경우 결측행 삭제 진행

In [236]:
drop_index = drop_index + X_train['Electrical'][X_train['Electrical'].isna()].index.tolist()

In [237]:
drop_index

[656, 372, 924]

In [238]:
X_train = X_train.drop(index=drop_index, axis=0)
y_train = y_train.drop(index=drop_index, axis=0)

In [239]:
X_train.shape, y_train.shape

((1165, 81), (1165, 2))

결측치가 제대로 삭제됨을 확인

--------

# X_test에 대해서도 동일하게 진행

In [240]:
for i in X_test.columns:
    if X_test[i].isna().sum() != 0:
        print('{} : {}'.format(i, X_test[i].isna().sum()))

LotFrontage : 50
Alley : 276
MasVnrType : 1
MasVnrArea : 1
BsmtQual : 6
BsmtCond : 6
BsmtExposure : 6
BsmtFinType1 : 6
BsmtFinType2 : 6
FireplaceQu : 126
GarageType : 17
GarageYrBlt : 17
GarageFinish : 17
GarageQual : 17
GarageCond : 17
PoolQC : 289
Fence : 233
MiscFeature : 281


In [241]:
X_test = pd.merge(X_test, LotArea_LotFrontage_Ratio, how='left', on='LotShape')

In [242]:
X_test['LotFrontage'] = X_test['LotFrontage'].fillna(X_test['LotArea'] * X_test['Ratio'])

In [243]:
drop_index = []

In [244]:
cols = ['MasVnrType', 'MasVnrArea']
temp3 = []
for i in range(len(cols)):
    if i+1 == len(cols):
        print(temp3)
        break
    temp1 = X_test[cols[i]][X_test[cols[i]].isna()].index
    temp2 = X_test[cols[i+1]][X_test[cols[i+1]].isna()].index
    for i in temp2:
        if i not in temp1:
            temp3.append(i)
            drop_index.append(i)

[]


In [245]:
cols = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']
temp3 = []
for i in range(len(cols)):
    if i+1 == len(cols):
        print(temp3)
        break
    temp1 = X_test[cols[i]][X_test[cols[i]].isna()].index
    temp2 = X_test[cols[i+1]][X_test[cols[i+1]].isna()].index
    for i in temp2:
        if i not in temp1:
            temp3.append(i)
            drop_index.append(i)

[]


In [246]:
cols = ['GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond']
temp3 = []
for i in range(len(cols)):
    if i+1 == len(cols):
        print(temp3)
        break
    temp1 = X_test[cols[i]][X_test[cols[i]].isna()].index
    temp2 = X_test[cols[i+1]][X_test[cols[i+1]].isna()].index
    for i in temp2:
        if i not in temp1:
            temp3.append(i)
            drop_index.append(i)

[]


In [247]:
drop_index

[]

In [248]:
X_test = X_test.drop(index=drop_index, axis=0)
y_test = y_test.drop(index=drop_index, axis=0)

In [249]:
X_test.shape, y_test.shape

((292, 81), (292, 2))

--------

# Label Encoding을 개별 데이터로 나눈 상태에서 진행하려 했으나 희소한 값 때문에 오류 발생.   
# 이를 방지하고자 X_train과 X_test를 concat하여 Ratio열을 삭제하고 Label Encoding 진행

In [250]:
all_data = pd.concat([X_train, X_test])

In [251]:
all_data = all_data.drop(columns='Ratio', axis=1)

In [252]:
col_int_float = all_data.select_dtypes(include=['int64', 'float64']).columns
col_object = all_data.select_dtypes(include='object').columns

for col in col_int_float:
  all_data[col] = all_data[col].fillna(0)
  all_data[col] = all_data[col].fillna(0)

for col in col_object:
  all_data[col] = all_data[col].fillna('Not_Included')
  all_data[col] = all_data[col].fillna('Not_Included')

In [253]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
for col in col_object:
    all_data[col] = LE.fit_transform(all_data[col])

In [254]:
for i in all_data.columns:
    if all_data[i].isna().sum() != 0:
        print('{} : {}'.format(i, all_data[i].isna().sum()))

-------

# 다시 데이터를 X_train과 X_test로 분리하고 id열 삭제

In [255]:
X_train = all_data[:len(X_train)]
X_test = all_data[len(X_train):]

In [256]:
X_train_id = X_train['id']
X_train = X_train.drop(columns='id', axis=1)
X_test_id = X_test['id']
X_test = X_test.drop(columns='id', axis=1)

-------

# 모델링
# 라벨 인코팅이 진행된 열이 많은 만큼 RandomForestRegressor 사용   
# RandomForestRegressor에 RandomizedSearchCV 적용

In [285]:
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {'n_estimators': randint(low=100, high=400), 'max_depth': randint(low=3, high=20), 'min_samples_split' : randint(low=2, high=30)}

model = RandomForestRegressor(random_state=2022)

rnd_search = RandomizedSearchCV(model, param_distributions=param_distribs, n_iter=10, scoring='neg_mean_squared_error', cv=5, random_state=2022)

rnd_search.fit(X_train, y_train['SalePrice'])

print(rnd_search.best_params_)
print('train : ', rnd_search.score(X_train, y_train['SalePrice']))
print('test : ', rnd_search.score(X_test, y_test['SalePrice']))

prediction = rnd_search.predict(X_test)
output = pd.DataFrame({'id' : X_test_id, 'SalePrice' : prediction})
output.to_csv('20220606.csv', index=False)

{'max_depth': 4, 'min_samples_split': 29, 'n_estimators': 111}
train :  -4867787777.590344
test :  -2903926027.0870795


--------

# RMSE 확인

In [286]:
from sklearn.metrics import mean_squared_error
RMSE = mean_squared_error(y_test['SalePrice'], output['SalePrice'])**0.5
print(RMSE)

53888.08798878542
